In [ ]:
!pip install memory-profiler


In [ ]:
import heapq
from collections import deque
from tkinter import Tk, Canvas
import time
from memory_profiler import profile
import random
import math
from itertools import count

class Pair:
    def _init_(self, first, second, parentF, parentS, depth, costFromStart=0):
        self.first = first
        self.second = second
        self.parentF = parentF
        self.parentS = parentS
        self.depth = depth
        self.costFromStart = costFromStart

    def _lt_(self, other):
        return self.costFromStart < other.costFromStart

class Maze:
  #0 represents open cells and -1 represents walls.
    def _init_(self, maze, startX, startY, endX, endY):
        self.maze = maze
        self.startX = startX
        self.startY = startY
        self.endX = endX
        self.endY = endY
        self.visited = [[False for _ in row] for row in maze]
        self.Xlength = len(maze)
        self.Ylength = len(maze[0])

    def visualize_maze(maze):
      cell_size = 100  # Size of each cell in the maze
      window_width = maze.Ylength * cell_size
      window_height = maze.Xlength * cell_size

      # Create the tkinter window
      window = Tk()
      window.title("Maze Visualization")
      canvas = Canvas(window, width=window_width, height=window_height, bg="white")
      canvas.pack()

      # Draw the maze
      for row in range(maze.Xlength):
          for col in range(maze.Ylength):
              x1, y1 = col * cell_size, row * cell_size
              x2, y2 = x1 + cell_size, y1 + cell_size
              if maze.maze[row][col] == -1:  # Wall
                  color = "black"
              elif (row, col) == (maze.startX, maze.startY):  # Start
                  color = "green"
              elif (row, col) == (maze.endX, maze.endY):  # Goal
                  color = "red"
              else:  # Free cell
                  color = "white"
              canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="gray")

      window.mainloop()



## prints the path in the terminal and return it
def print_solution(maze, solution, curr):
    path=[]
    if curr.parentF == maze.startX and curr.parentS == maze.startY:
        print(f"Start: ({curr.parentF}, {curr.parentS}) moved to ({curr.first}, {curr.second}) Depth: {curr.depth}")
        path.append((curr.first, curr.second))
        return path
    else:
        print(f"Next: ({curr.parentF}, {curr.parentS}) moved to ({curr.first}, {curr.second}) Depth: {curr.depth}")
        path.append((curr.first, curr.second))

        for pair in solution:
            if pair.first == curr.parentF and pair.second == curr.parentS:
                return print_solution(maze, solution, pair)
    return path

# Importing the necessary libraries


In [ ]:
import numpy as np
import random

# 1. State Space Representation


# here we are defining the state space and identifing the initial and goal states by Creates a list of all possible states in the grid and each state is represented as a tuple (x, y)


In [ ]:
state_space = [(x, y) for x in range(5) for y in range(5)]


initial_state: The starting point of the agent (top-left corner of the grid)




goal_state: The target the agent is trying to reach (bottom-right corner)


In [ ]:
initial_state = (0, 0)
goal_state = (4, 4)


# 2. Action Set


In [ ]:
actions = ['up', 'down', 'left', 'right']


#calculates the next state based on the current state and the chosen action to nsures the agent doesn’t move outside the boundaries of the grid (0 <= x, y <= 99).

In [ ]:
def get_next_state(state, action):
    x, y = state
    if action == 'up':
        return (max(x - 1, 0), y)  # prevents moving out of the grid
    elif action == 'down':
        return (min(x + 1, 4), y)  # prevents moving out of the grid
    elif action == 'left':
        return (x, max(y - 1, 0))  # prevents moving out of the grid
    elif action == 'right':
        return (x, min(y + 1, 4))  # prevents moving out of the grid
    return state


# 3. Reward Function


In [ ]:
def reward_function(state):
    if state == goal_state:
        return 100
    else:
        return -1

# 4. Q-Learning Setup


# here we are createing a Q-table with dimensions which is [number of states, number of actions] in the begging all Q-values are set to 0 (cuz agent has no knowledge).

In [ ]:
q_table = np.zeros((len(state_space), len(actions)))


In [ ]:
learning_rate = 0.1
discount_factor = 0.9
epsilon = 1.0  # exploration rate
epsilon_decay = 0.995
min_epsilon = 0.1

# maps each state to a unique index for easy access in the Q-table

In [ ]:
state_to_index = {state: i for i, state in enumerate(state_space)}


# train agent function

# This function trains the agent using the Q-learning algorithm that we used. The agent starts at the initial state and learns over a specified number of episodes. In each episode, the agent explores or exploits its knowledge by either choosing a random action (exploration) or selecting the best-known action based on the Q-table (exploitation). After performing an action, it moves to the next state, receives a reward based on the environment, and updates the Q-value for the action it just took. If the agent reaches the goal, the episode ends early. The exploration rate (`epsilon`) is reduced gradually after each episode to encourage the agent to exploit its learned policy more as training progresses. This process helps the agent learn the optimal actions to take in different states to maximize its rewards.

In [ ]:
def train_agent(episodes=1000, max_steps=100):
    global epsilon
    for episode in range(episodes):
        state = initial_state
        for step in range(max_steps):
            state_index = state_to_index[state]


            if random.uniform(0, 1) < epsilon:
                action_index = random.choice(range(len(actions)))
            else:
                action_index = np.argmax(q_table[state_index])

            action = actions[action_index]
            next_state = get_next_state(state, action)
            next_state_index = state_to_index[next_state]

            reward = reward_function(next_state)
            q_table[state_index, action_index] = q_table[state_index, action_index] + learning_rate * (
                reward + discount_factor * np.max(q_table[next_state_index]) - q_table[state_index, action_index]
            )

            state = next_state

            if state == goal_state:
                break

        epsilon = max(min_epsilon, epsilon * epsilon_decay)

In [ ]:
train_agent()

# 6. Performance Monitoring


# printing the Q-values to evaluate learning progress


In [ ]:
def print_q_values():
    print("Q-values for each state:")
    for state, index in state_to_index.items():
        print(f"State {state}: {q_table[index]}")

# Display learned Q-values


In [ ]:
print_q_values()

Q-values for each state:
State (0, 0): [36.7596504  41.82425386 37.19503077 42.612659  ]
State (0, 1): [42.25455522 48.45851    37.21829169 48.30274241]
State (0, 2): [34.59690251 54.95198828 30.77842598 32.3881782 ]
State (0, 3): [23.1967254  61.90030841 15.94536819 31.23382687]
State (0, 4): [23.8416263  63.88178878 15.05827629 20.17989616]
State (1, 0): [16.75575287 12.98700734 19.65860255 48.44595183]
State (1, 1): [42.15863745 54.38363878 41.7992386  54.9539    ]
State (1, 2): [47.98768393 61.95747386 47.90763991 62.171     ]
State (1, 3): [53.35949022 70.19       54.615209   69.94252133]
State (1, 4): [41.93632991 79.09917055 52.98150109 61.41387367]
State (2, 0): [11.01554325  0.36629211 10.32206737 32.60930734]
State (2, 1): [27.52713824 15.16864006  7.16700251 62.08028891]
State (2, 2): [47.68233971 45.58498257 35.39645176 70.18996536]
State (2, 3): [61.79145038 78.3222658  61.75881702 79.1       ]
State (2, 4): [70.09079487 89.         70.12319432 79.08447098]
State (3, 0): [

# Analyzing the policy learned by the agent


In [ ]:
def analyze_policy():
    print("\nLearned Policy:")
    for state, index in state_to_index.items():
        best_action = actions[np.argmax(q_table[index])]
        print(f"State {state}: Best action -> {best_action}")

# Display the learned policy


In [ ]:
analyze_policy()


Learned Policy:
State (0, 0): Best action -> right
State (0, 1): Best action -> down
State (0, 2): Best action -> down
State (0, 3): Best action -> down
State (0, 4): Best action -> down
State (1, 0): Best action -> right
State (1, 1): Best action -> right
State (1, 2): Best action -> right
State (1, 3): Best action -> down
State (1, 4): Best action -> down
State (2, 0): Best action -> right
State (2, 1): Best action -> right
State (2, 2): Best action -> right
State (2, 3): Best action -> right
State (2, 4): Best action -> down
State (3, 0): Best action -> right
State (3, 1): Best action -> right
State (3, 2): Best action -> right
State (3, 3): Best action -> down
State (3, 4): Best action -> down
State (4, 0): Best action -> right
State (4, 1): Best action -> right
State (4, 2): Best action -> right
State (4, 3): Best action -> right
State (4, 4): Best action -> up
